# imports

In [ ]:
import os
import os.path
import pickle
import matplotlib.pyplot as plt
from typing import Callable

from test_runner import *
from test_runner.translators import *
from test_runner.analysers import SearchResult


# from test_runner import TestCase, BaseTestRunner, LiftedPlanningRunner, GroundedPlanningRunner
# from test_runner.tapaal_caller import QueryResult

from parse_results import translator_result_type, search_result_type#, load_translator_results, load_search_results, 


# Load Data

In [ ]:


results_dir = "./results"
plot_save_dir = "./results/plots"

os.makedirs(plot_save_dir, exist_ok=True)


results_path = "./results"

def load_translator_results() -> translator_result_type:
    with open(os.path.join(results_path, f"translator_results.pickle"), "rb") as f:
        return pickle.load(f)


def load_search_results() -> search_result_type:
    with open(os.path.join(results_path, f"search_results.pickle"), "rb") as f:
        return pickle.load(f)
    

translator_results: translator_result_type = load_translator_results()
search_results: search_result_type = load_search_results()

# searcher -> translator -> test_case
search_results_search_translator_test_case: dict["BaseSearcher", dict["BaseTranslator", dict["TestCase", list["SearchResult"]]]] = dict()
for translator, translator_results in search_results.items():
    for test_case, test_results in translator_results.items():
        for search, results in test_results.items():
            search_results_search_translator_test_case[search] = search_results_search_translator_test_case.get(search, dict())
            search_results_search_translator_test_case[search][translator] = search_results_search_translator_test_case[search].get(translator, dict())
            search_results_search_translator_test_case[search][translator][test_case] = results


# test_case _> searcher -> translator
search_results_test_case_searcher_translator: dict["TestCase", dict["BaseSearcher", dict["BaseTranslator", list["SearchResult"]]]] = dict()
for translator, translator_results in search_results.items():
    for test_case, test_results in translator_results.items():
        for search, results in test_results.items():
            search_results_test_case_searcher_translator[test_case] = search_results_test_case_searcher_translator.get(test_case, dict())
            search_results_test_case_searcher_translator[test_case][search] = search_results_test_case_searcher_translator[test_case].get(search, dict())
            search_results_test_case_searcher_translator[test_case][search][translator] = results



In [ ]:
results_dir = "./results"
plot_save_dir = "./results/plots"

os.makedirs(plot_save_dir, exist_ok=True)


results: dict[TestCase, dict[BaseTestRunner, list[QueryResult]]] = {}

for filename in os.listdir(results_dir):
    print(f"Loading {filename}")
    full_path = os.path.join(results_dir, filename)
    if os.path.isfile(full_path):
        with open(full_path, "rb") as f:
            data: dict[TestCase, dict[BaseTestRunner, list[QueryResult]]] = pickle.load(f)
            for k,v in data.items():
                results[k] = v

results

# Plots

## General Setup

## Cactus Plot

In [ ]:
# System + User Time

def make_cactus_plot(name: str, description: str, reduction: Callable[["SearchResult"], float]):
    fig_rows = 4
    fig_cols = 3


    i=1
    plt.figure(figsize=(20, 12))
    for (test_case, test_case_results) in search_results_test_case_searcher_translator.items():
        plt.subplot(fig_rows, fig_cols, i)
        for (searcher, searcher_results) in test_case_results.items():
            for (translator, results) in searcher_results.items():
                times = [reduction(res) for res in results]
                times.sort()
                median = times[int(len(times)/2)]
                plt.plot(times, 'o-', label=f"{translator.name}({searcher.name}) - median={median:.4f}")

        plt.title(f'{description} - {test_case.name}')
        plt.gca().legend(loc='best')
        plt.xlabel('Index')
        plt.ylabel('Time (sec)')
        i += 1

    plt.savefig(os.path.join(plot_save_dir, f"{name}.png"))

make_cactus_plot("total_time", "System + User time", lambda res: res.time.seconds_system + res.time.seconds_user)


In [ ]:
# Total Time

fig_rows = 2
fig_cols = 3

i=1
plt.figure(figsize=(10, 6))
for test_case, runners in results.items():
    plt.subplot(fig_rows, fig_cols, i)
    for runner, query_results in runners.items():
        # times = [res.time_total for res in query_results]
        times = [res.time_total for res in query_results]
        times.sort()
        median = times[int(len(times)/2)]
        plt.plot(times, 'o-', label=f"{runner.translation_name}")  # - {runner.description} - median={median:.4f}")

    plt.title(f'Total Time - {test_case.name}')
    plt.gca().legend(loc='best')
    plt.xlabel('Index')
    plt.ylabel('Time (sec)')

    #Saving the plots
    # filename = f'plots/{runner}_{test_case}.png'
    # plt.savefig(filename)
    i += 1

plt.savefig(os.path.join(plot_save_dir, "blocksworld_total_time.png"))

In [ ]:
# Tapaal Time

fig_rows = 2
fig_cols = 3

i=1
plt.figure(figsize=(10, 6))
for test_case, runners in results.items():
    plt.subplot(fig_rows, fig_cols, i)
    for runner, query_results in runners.items():
        times = [res.time_tapaal for res in query_results]
        times.sort()
        median = times[int(len(times)/2)]
        plt.plot(times, 'o-', label=f"{runner.translation_name}")  # - {runner.description} - median={median:.4f}")

    plt.title(f'Tapaal Time - {test_case.name}')
    plt.gca().legend(loc='best')
    plt.xlabel('Index')
    plt.ylabel('Time (sec)')

    #Saving the plots
    # filename = f'plots/{runner}_{test_case}.png'
    # plt.savefig(filename)
    i += 1

plt.savefig(os.path.join(plot_save_dir, "blocksworld_tapaal_time.png"))

In [ ]:
# Verification Time

fig_rows = 2
fig_cols = 3

i=1
plt.figure(figsize=(10, 6))
for test_case, runners in results.items():
    plt.subplot(fig_rows, fig_cols, i)
    for runner, query_results in runners.items():
        times = [res.output["time_verification"] for res in query_results]
        times.sort()
        median = times[int(len(times)/2)]
        plt.plot(times, 'o-', label=f"{runner.translation_name}")  # - {runner.description} - median={median:.4f}")

    plt.title(f'Verification Time - {test_case.name}')
    plt.gca().legend(loc='best')
    plt.xlabel('Index')
    plt.ylabel('Time (sec)')

    #Saving the plots
    # filename = f'plots/{runner}_{test_case}.png'
    # plt.savefig(filename)
    i += 1

# plt.show()
plt.savefig(os.path.join(plot_save_dir, "blocksworld_verification_time.png"))

In [ ]:
# Translation Time

fig_rows = 2
fig_cols = 3

i=1
plt.figure(figsize=(10, 6))
for test_case, runners in results.items():
    plt.subplot(fig_rows, fig_cols, i)
    for runner, query_results in runners.items():
        times = [res.time_translation for res in query_results]
        times.sort()
        median = times[int(len(times)/2)]
        plt.plot(times, 'o-', label=f"{runner.translation_name}")  # - {runner.description} - median={median:.4f}")

    plt.title(f'Translation Time - {test_case.name}')
    plt.gca().legend(loc='best')
    plt.xlabel('Index')
    plt.ylabel('Time (sec)')

    #Saving the plots
    # filename = f'plots/{runner}_{test_case}.png'
    # plt.savefig(filename)
    i += 1

# plt.show()
plt.savefig(os.path.join(plot_save_dir, "blocksworld_translation_time.png"))